<div class="alert alert-success">

The <a href="https://www.cato.org/human-freedom-index/2021 ">Human Freedom Index</a> measures economic freedoms such as the freedom to trade or to use sound money, and it captures the degree to which people are free to enjoy the major freedoms often referred to as civil liberties—freedom of speech, religion, association, and assembly— in the countries in the survey. In addition, it includes indicators on rule of law, crime and violence, freedom of movement, and legal discrimination against same-sex relationships. We also include nine variables pertaining to women-specific freedoms that are found in various categories of the index.

<u>Citation</u>

Ian Vásquez, Fred McMahon, Ryan Murphy, and Guillermina Sutter Schneider, The Human Freedom Index 2021: A Global Measurement of Personal, Civil, and Economic Freedom (Washington: Cato Institute and the Fraser Institute, 2021).
    
</div>


## Using simple MLP CLassifier from sklearn for the first part

In [10]:
import numpy as np
import pandas as pd

In [11]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("https://github.com/jnin/information-systems/raw/main/data/hfi_cc_2021.csv")
df.drop(["year", "ISO", "countries"], axis = 1, inplace = True)

columns_rank = list(columns for columns in df if "rank" in columns)
df.drop(columns_rank, axis = 1, inplace = True)

columns_score = list(columns for columns in df if "score" in columns)
df.drop(columns_score, axis = 1, inplace = True)

df.dropna(subset = ["hf_quartile"], inplace = True)

X = df.drop(["hf_quartile"], axis = 1)
y = df[["hf_quartile"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75)

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

categorical_values = X_train.select_dtypes(include = ["object"]).columns.tolist()

numerical_values = X_train.select_dtypes(exclude = ["object"]).columns.tolist()

transformer = ColumnTransformer([("encode", OneHotEncoder(),categorical_values)], remainder = "passthrough")

pipe = Pipeline([("encoder", transformer), 
                 ("imputer", SimpleImputer(strategy = "most_frequent")), 
                 ("scaler", StandardScaler()), 
                 ("neural_model", MLPClassifier(max_iter = 250))])

In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "neural_model__learning_rate_init" : [0.001, 0.0001],
    "neural_model__alpha" : [0.0001, 1]
}

grid = GridSearchCV(pipe, param_grid, cv = 3)

grid.fit(X_train, y_train.values.ravel())

training_score = grid.best_score_

C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_

In [14]:
best_model = grid.best_estimator_.fit(X_train, y_train.values.ravel())

score = best_model.score(X_test, y_test)

# Comparing the training and test scores to check for overfitting
print('Training score: ',{training_score})
print('Test score: ', {score}) 

# It does not seem like the model is overfitting as the training and test scores are very similar

Training score:  {0.9357172834854319}
Test score:  {0.9571734475374732}


C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


## Using Keras Neural Networks to create a Sequential model

#### Importing the necessary libraries

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.decomposition import PCA
from scikeras.wrappers import KerasClassifier
from keras.regularizers import l2, l1
from keras_tuner import HyperParameters
from keras_tuner.tuners import RandomSearch
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.layers import LeakyReLU
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

#### Preparing the dataset


*   Loading the dataset into a dataframe and dropping irrelevant columns
*   Removing rows where the target variable (hf_quartile) is missing
*   Splitting the dataframe into independent and dependent (target) variables
*   Splitting the datasets into training and test sets 

In [16]:
from sklearn.model_selection import train_test_split

# Loading the dataset and dropping irrelevant columns, and columns with rank or score 
df = pd.read_csv("https://github.com/jnin/information-systems/raw/main/data/hfi_cc_2021.csv")
df.drop(["year", "ISO", "countries"], axis = 1, inplace = True)

columns_rank = list(columns for columns in df if "rank" in columns)
df.drop(columns_rank, axis = 1, inplace = True)

columns_score = list(columns for columns in df if "score" in columns)
df.drop(columns_score, axis = 1, inplace = True)

# Dropping rows in which the target variable value is missing
df.dropna(subset = ["hf_quartile"], inplace = True)

# Separating the independent and dependent (target) variables
X = df.drop(["hf_quartile"], axis = 1)
y = df[["hf_quartile"]]

# Splitting the dataset into train and test sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.80, random_state = 123) 
# Different random_state / seeds and a different train / test split sizes were tested to check if they improve the model's performance

#### **Defining a neural network model using Keras library with 4 fully connected layers**

*   **Layer 1:** 50 neurons, LeakyReLU activation, and L2 regularization
*   **Layers 2:** 30 neurons, LeakyReLU activation and L2 regularization
*   **Layer 3:** 10 neurons, LeakyReLU activation and L2 regularization
*   **Layer 4:** 4 units and a softmax activation function

**Notes about the network architecture:**

- We start with 50 neurons in the first layer and decrease with each layer to gradually reduce the complexity of the model and prevent overfitting. We experimented with different sizes such as starting with 100 and decreasing by 20 per layer or starting with 200 and decreasing by 50 per layer and checked accuracy / overfitting with each combination until we finally decided on these sizes through trial and error.

- LeakyReLU is the activation function used in layers 1-3  as it is known to perform well in deep neural networks. We tried increasing the alpha from the common practice of 0.1 to 0.2 to increase regularization (and therefore reduce chances of overfitting).

- L2 regularization is used in layers 1-3 to prevent overfitting by adding a penalty term to the loss function that encourages the model to have smaller weights.

- Dropout layers are used as an additional regularization technique that randomly drops out some of the neurons in the network during training, which helps further prevent overfitting. Here, a dropout rate of 20% is used. We also experimented with this parameter starting with 10% then increasing it to reduce overfitting.

- Softmax is the activation function used in the final layer because it can map the model's final outputs to probabilities that sum to 1 making it ideal for multi-class classification problems such as this one.

- The model is compiled using the Adam optimizer with a learning rate of 0.001, sparse categorical crossentropy loss, and accuracy as the evaluation metric. This neural network model is designed for multi-class classification problems, where there are four possible output classes (in this case a Human Freedom Quartile of 1, 2, 3 or 4). 

In [17]:
# Define a Sequential model object
model = Sequential()

## Layer 1
# Adding a fully connected layer with 100 units, LeakyReLU activation with alpha value of 0.1, and L2 regularization with a strength of 0.001
# The input dimension of the layer is 122 to match the number of featurs in the dataset
model.add(Dense(units = 50, activation = LeakyReLU(alpha = 0.2), input_dim = (122), kernel_regularizer = l2(0.001)))

# Adding a dropout layer with 10% dropout rate to avoid overfitting
model.add(Dropout(0.2)) 


## Layer 2
# Adding another fully connected layer with 80 units, LeakyReLU activation with alpha value of 0.1, and L2 regularization with a strength of 0.001
model.add(Dense(units = 30, activation = LeakyReLU(alpha = 0.2), kernel_regularizer = l2(0.001)))

# Adding a dropout layer with 10% dropout rate to avoid overfitting
model.add(Dropout(0.2))


## Layer 3
# Adding another fully connected layer with 60 units, LeakyReLU activation with alpha value of 0.1, and L2 regularization with a strength of 0.001
model.add(Dense(units = 10, activation = LeakyReLU(alpha = 0.2), kernel_regularizer = l2(0.001)))

# Adding a dropout layer with 10% dropout rate to avoid overfitting
model.add(Dropout(0.2))


## Layer 4
# Adding a final fully connected layer with 4 units and softmax activation
# This layer outputs probabilities for each of the 4 output classes
model.add(Dense(units = 4, activation = "softmax"))

# Compiling the model with Adam optimizer with a learning rate of 0.001, sparse categorical crossentropy loss, and accuracy as the evaluation metrica
model.compile(optimizer = Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

#### Creating a Pipeline consisting of a SimpleImputer with the most frequent strategy, a OneHotEncoder for the categorical variables, a standard scaler, and a KerasClassifier model specifying 65 epochs and batch sizes of 100.

The number of epochs and batch sizes were decided through trial and error of different values until the best balance was reach between accuracy, (lack of) overfitting and compute time.

In [18]:
# Identifying the categorical features to be encoded
categorical_values = X_train.select_dtypes(include = ["object"]).columns.tolist()

numerical_values = X_train.select_dtypes(exclude = ["object"]).columns.tolist()

# Applying one-hot encoding to categorical features
transformer = ColumnTransformer([("encode", OneHotEncoder(),categorical_values)], remainder = "passthrough")

# Applying the early stopping technique to stop training if there is no improvement in loss for 10 epochs, and restore_best_weights=True to restore the best model weights
early_stop = EarlyStopping(monitor = 'loss', patience=10, restore_best_weights=True)

# Defining the steps for preprocessing the data and training the neural network with scikit-learn and Keras
pipe2 = Pipeline([("encoder", transformer), 
                 ("imputer", SimpleImputer(strategy = "most_frequent")), 
                 ("scaler", StandardScaler()),
                 ("model", KerasClassifier(build_fn = model, epochs = 65, batch_size = 100, callbacks=[early_stop]))])
# Specifying the training parameters (batch sizes and number of epochs)

### Fitting the model using the created pipeline

In [19]:
pipe2.fit(X_train, y_train)

Epoch 1/65


C:\Users\Lenovo\anaconda3\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


15/15 [==============================] - 1s 2ms/step - loss: 1.5277 - accuracy: 0.3831
Epoch 2/65
15/15 [==============================] - 0s 2ms/step - loss: 1.1397 - accuracy: 0.5405
Epoch 3/65
15/15 [==============================] - 0s 2ms/step - loss: 0.9396 - accuracy: 0.6417
Epoch 4/65
15/15 [==============================] - 0s 2ms/step - loss: 0.8792 - accuracy: 0.6758
Epoch 5/65
15/15 [==============================] - 0s 2ms/step - loss: 0.8025 - accuracy: 0.7200
Epoch 6/65
15/15 [==============================] - 0s 2ms/step - loss: 0.7291 - accuracy: 0.7488
Epoch 7/65
15/15 [==============================] - 0s 2ms/step - loss: 0.6590 - accuracy: 0.7904
Epoch 8/65
15/15 [==============================] - 0s 2ms/step - loss: 0.6345 - accuracy: 0.7944
Epoch 9/65
15/15 [==============================] - 0s 2ms/step - loss: 0.5718 - accuracy: 0.8265
Epoch 10/65
15/15 [==============================] - 0s 2ms/step - loss: 0.5673 - accuracy: 0.8131
Epoch 11/65
15/15 [===========

Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encode', OneHotEncoder(),
                                                  ['region'])])),
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler()),
                ('model',
                 KerasClassifier(batch_size=100, build_fn=<keras.engine.sequential.Sequential object at 0x00000171FBFDFE20>, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000171FC16D730>], epochs=65))])

#### Checking the accuracy score
The best achieved score was 95.99%

In [20]:
final_score = pipe2.score(X_test, y_test)
print(final_score)

4/4 [==============================] - 0s 2ms/step
0.9598930481283422


In [21]:
# Comparing the training and test scores to check for overfitting
print('Training score: ',pipe2.score(X_train, y_train))
print('Test score: ', {final_score}) 

# It does not seem like the model is overfitting as the training and test scores are similar

15/15 [==============================] - 0s 719us/step
Training score:  0.9866041527126591
Test score:  {0.9598930481283422}
